# Quick start

## Starting the server

### Install `lavender-data`

You can install lavender-data using pip.

```sh
pip install lavender-data
```

### Run the server

You can run the lavender data server simply with following command:
```sh
python3 -m lavender_data.server --port 8000 --host 0.0.0.0
```

The server will be running on http://0.0.0.0:8000.
This is obviously a blocking call, so we'll spawn a subprocess instead and move on.

In [27]:
import select
import subprocess

command = "python3 -m lavender_data.server --port 8000 --host 0.0.0.0"

server_process = subprocess.Popen(
    command.split(),
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE,
)

server_ready = False
while server_process.poll() is None and not server_ready:
    read_fds, _, _ = select.select([server_process.stdout, server_process.stderr], [], [], 1)
    for fd in read_fds:
        line = fd.readline().decode().strip()
        if "Application startup complete." in line:
            server_ready = True
        print(line)

INFO:     Started server process [12086]
INFO:     Waiting for application startup.
[2025-03-25 10:46:54,122] INFO - lavender_data.server.services.registries: Imported modules/lavender_data_modules.py: {'preprocessor': 'append_new_column', 'filter': 'only_even_uids', 'collater': 'pylist'}
INFO:     Application startup complete.


## Using the client

### Initialize the client

Use `lavender_data.client.api.init` to initialize the client.

In [29]:
from lavender_data.client import api as lavender

lavender.init(api_url="http://localhost:8000")

Let's check if we're connected by listing the datasets with `get_datasets`.

In [30]:
lavender.get_datasets()

[]

### Define the schema

Make a new dataset with `create_dataset`. `uid_column_name` is the name of the column that will be used as the unique identifier for each sample.

In [31]:
lavender.create_dataset(name="test-dataset", uid_column_name="uid")

DatasetPublic(name='test-dataset', created_at=datetime.datetime(2025, 3, 25, 1, 47, 34), id='ds-m8nu5wgbqhfqec93hc7t', uid_column_name='uid', additional_properties={})

In [32]:
dataset = lavender.get_dataset(name="test-dataset")
dataset

GetDatasetResponse(name='test-dataset', created_at=datetime.datetime(2025, 3, 25, 1, 47, 34), columns=[], shardsets=[], id='ds-m8nu5wgbqhfqec93hc7t', uid_column_name='uid', additional_properties={})

Add a shardset to the dataset with `create_shardset`.
Let's add 2 columns, `uid` and `text`.

In [33]:
shardset = lavender.create_shardset(
    dataset_id=dataset.id,
    location="file://.cache/lavender-data/test_shards",
    columns=[
        lavender.DatasetColumnOptions(
            name="uid",
            description="Unique identifier",
            type_="int",
        ),
        lavender.DatasetColumnOptions(
            name="text",
            description="A text field",
            type_="str",
        ),
    ],
)
shardset

CreateShardsetResponse(dataset_id='ds-m8nu5wgbqhfqec93hc7t', location='file://.cache/lavender-data/test_shards', created_at=datetime.datetime(2025, 3, 25, 1, 47, 36), columns=[DatasetColumnPublic(dataset_id='ds-m8nu5wgbqhfqec93hc7t', shardset_id='ss-m8nu5y3agbftxmtrj0rs', name='uid', type_='int', created_at=datetime.datetime(2025, 3, 25, 1, 47, 36), id='dc-m8nu5y3cd6vxfgkp6188', description='Unique identifier', additional_properties={}), DatasetColumnPublic(dataset_id='ds-m8nu5wgbqhfqec93hc7t', shardset_id='ss-m8nu5y3agbftxmtrj0rs', name='text', type_='str', created_at=datetime.datetime(2025, 3, 25, 1, 47, 36), id='dc-m8nu5y3cwrspfzo63rxi', description='A text field', additional_properties={})], id='ss-m8nu5y3agbftxmtrj0rs', shard_count=0, total_samples=0, additional_properties={})

Now the dataset has 2 columns, `uid` and `text`.

In [34]:
[f"{col.name} ({col.type_}): {col.description}" for col in lavender.get_dataset(dataset.id).columns]

['text (str): A text field', 'uid (int): Unique identifier']

### Write shards

Use `Writer` to write shards to the shardset. Specify desired file format (supported: csv, parquet), and the id of the dataset and shardset.

In [35]:
from lavender_data.shard import Writer

writer = Writer.get(
    format="csv",
    dataset_id=dataset.id,
    shardset_id=shardset.id,
)

`Writer.write` takes a `samples` and `shard_index`.

`samples` is a list of dictionaries, where the keys are the column names and the values are the data.
These samples will be written to the shard file which will be stored at the location specified in `shardset`.

`shard_index` is the index of the shard file to write to.
Shards will be ordered by the `shard_index` during the iteration.

Let's make 10 shards with 100 samples each.

In [36]:
shard_count = 10
samples_per_shard = 10

for shard_index in range(shard_count):
    samples = [
        {
            "text": f"Sample {i + shard_index * samples_per_shard}",
            "uid": i + shard_index * samples_per_shard,
        } for i in range(samples_per_shard)
    ]
    writer.write(
        samples=samples,
        shard_index=shard_index,
    )

Now the shardset has 100 samples.

In [37]:
shardset = lavender.get_dataset(dataset.id).shardsets[0]
print(f"Shard count: {shardset.shard_count}, Total samples: {shardset.total_samples}")

Shard count: 10, Total samples: 100


### Add a new column

You might want to add a new feature to the dataset. In this case, you can add a new column to the dataset by adding a new shardset.

Be aware that all the shardsets must have the `uid_column_name` column.

In [38]:
new_shardset = lavender.create_shardset(
    dataset_id=dataset.id,
    location="file://.cache/lavender-data/test_shards_new",
    columns=[
        lavender.DatasetColumnOptions(
            name="uid",
            description="Unique identifier",
            type_="int",
        ),
        lavender.DatasetColumnOptions(
            name="new_text",
            description="A new text field",
            type_="str",
        ),
    ],
)

In [39]:
[f"{col.name} ({col.type_}): {col.description}" for col in lavender.get_dataset(dataset.id).columns]

['new_text (str): A new text field',
 'text (str): A text field',
 'uid (int): Unique identifier']

We'll add only 8 samples per shard this time, to demonstrate what happens when shardsets in the same dataset have different number of samples.


> For each sample, the shard index of the sample MUST be the same across all the shardsets.
> If not, it's hard to determine which shard the sample belongs to.
>
> For example, let's say you have 10 samples per shard in shardset A.
> Then, the 11th sample in shardset A belongs to 2nd shard.
> Let's say you derived a new shardset B from A, and had to drop 9th, 10th samples in A.
> Even though you dropped 2 samples, the 11th sample in shardset B should still belongs to 2nd shard.

In [40]:
writer = Writer.get(
    format="csv",
    dataset_id=dataset.id,
    shardset_id=new_shardset.id,
    persist_files=True,
)

shard_count = 10
new_samples_per_shard = 8

for shard_index in range(shard_count):
    samples = [
        {
            "new_text": f"Sample {i + shard_index * samples_per_shard}",
            "uid": i + shard_index * samples_per_shard,
        } for i in range(new_samples_per_shard)
    ]
    writer.write(
        samples=samples,
        shard_index=shard_index,
    )

In [41]:
new_shardset = lavender.get_dataset(dataset.id).shardsets[1]
print(f"Shard count: {new_shardset.shard_count}, Total samples: {new_shardset.total_samples}")

Shard count: 10, Total samples: 80


### Iterate over the dataset

Use `Iteration` to iterate over the dataset. Specify the dataset id and shardsets you want to iterate over.

Excluded shardsets will not be loaded. This can reduce huge amount of the overhead.

Best practice would be selecting only the shardsets you need. For example, let's say you have an image dataset, and you preprocessed the images into embeddings. Store the embeddings in a new shardset, and do not select it on iteration if you don't need it.

In [42]:
from lavender_data.client import Iteration

iteration = Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
)
iteration

In [43]:
for sample in iteration:
    print(sample)


{'text': 'Sample 0', 'uid': 0, '_lavender_data_indices': 0}
{'text': 'Sample 1', 'uid': 1, '_lavender_data_indices': 1}
{'text': 'Sample 2', 'uid': 2, '_lavender_data_indices': 2}
{'text': 'Sample 3', 'uid': 3, '_lavender_data_indices': 3}
{'text': 'Sample 4', 'uid': 4, '_lavender_data_indices': 4}
{'text': 'Sample 5', 'uid': 5, '_lavender_data_indices': 5}
{'text': 'Sample 6', 'uid': 6, '_lavender_data_indices': 6}
{'text': 'Sample 7', 'uid': 7, '_lavender_data_indices': 7}
{'text': 'Sample 8', 'uid': 8, '_lavender_data_indices': 8}
{'text': 'Sample 9', 'uid': 9, '_lavender_data_indices': 9}
{'text': 'Sample 10', 'uid': 10, '_lavender_data_indices': 10}
{'text': 'Sample 11', 'uid': 11, '_lavender_data_indices': 11}
{'text': 'Sample 12', 'uid': 12, '_lavender_data_indices': 12}
{'text': 'Sample 13', 'uid': 13, '_lavender_data_indices': 13}
{'text': 'Sample 14', 'uid': 14, '_lavender_data_indices': 14}
{'text': 'Sample 15', 'uid': 15, '_lavender_data_indices': 15}
{'text': 'Sample 16', 

The samples will be shuffled if `shuffle` is set to `True`. You can fix the shuffled order by setting `shuffle_seed` to a fixed value.

`shuffle_block_size` is the number of shards to shuffle at a time. Larger value means more disk usage but gives more randomness.

In [44]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
    shuffle=True,
    shuffle_seed=42,
    shuffle_block_size=3,
):
    print(sample)

{'text': 'Sample 27', 'uid': 27, '_lavender_data_indices': 27}
{'text': 'Sample 15', 'uid': 15, '_lavender_data_indices': 15}
{'text': 'Sample 23', 'uid': 23, '_lavender_data_indices': 23}
{'text': 'Sample 17', 'uid': 17, '_lavender_data_indices': 17}
{'text': 'Sample 8', 'uid': 8, '_lavender_data_indices': 8}
{'text': 'Sample 9', 'uid': 9, '_lavender_data_indices': 9}
{'text': 'Sample 28', 'uid': 28, '_lavender_data_indices': 28}
{'text': 'Sample 24', 'uid': 24, '_lavender_data_indices': 24}
{'text': 'Sample 12', 'uid': 12, '_lavender_data_indices': 12}
{'text': 'Sample 0', 'uid': 0, '_lavender_data_indices': 0}
{'text': 'Sample 4', 'uid': 4, '_lavender_data_indices': 4}
{'text': 'Sample 16', 'uid': 16, '_lavender_data_indices': 16}
{'text': 'Sample 5', 'uid': 5, '_lavender_data_indices': 5}
{'text': 'Sample 13', 'uid': 13, '_lavender_data_indices': 13}
{'text': 'Sample 11', 'uid': 11, '_lavender_data_indices': 11}
{'text': 'Sample 22', 'uid': 22, '_lavender_data_indices': 22}
{'text'

The samples will be batched if `batch_size` is set.

In [45]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
    batch_size=10,
):
    print(sample)

{'text': ['Sample 0', 'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 8', 'Sample 9'], 'uid': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], '_lavender_data_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'text': ['Sample 10', 'Sample 11', 'Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16', 'Sample 17', 'Sample 18', 'Sample 19'], 'uid': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], '_lavender_data_indices': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
{'text': ['Sample 20', 'Sample 21', 'Sample 22', 'Sample 23', 'Sample 24', 'Sample 25', 'Sample 26', 'Sample 27', 'Sample 28', 'Sample 29'], 'uid': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], '_lavender_data_indices': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]}
{'text': ['Sample 30', 'Sample 31', 'Sample 32', 'Sample 33', 'Sample 34', 'Sample 35', 'Sample 36', 'Sample 37', 'Sample 38', 'Sample 39'], 'uid': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39], '_lavender_data_indices': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

### What happens if shardsets have different number of samples?

If shardsets have different number of samples, only the samples with all the columns will be loaded.


In [46]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id, new_shardset.id],
    batch_size=10,
):
    print(sample)

{'new_text': ['Sample 0', 'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 10', 'Sample 11'], 'uid': [0, 1, 2, 3, 4, 5, 6, 7, 10, 11], 'text': ['Sample 0', 'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 10', 'Sample 11'], '_lavender_data_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'new_text': ['Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16', 'Sample 17', 'Sample 20', 'Sample 21', 'Sample 22', 'Sample 23'], 'uid': [12, 13, 14, 15, 16, 17, 20, 21, 22, 23], 'text': ['Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16', 'Sample 17', 'Sample 20', 'Sample 21', 'Sample 22', 'Sample 23'], '_lavender_data_indices': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
{'new_text': ['Sample 24', 'Sample 25', 'Sample 26', 'Sample 27', 'Sample 30', 'Sample 31', 'Sample 32', 'Sample 33', 'Sample 34', 'Sample 35'], 'uid': [24, 25, 26, 27, 30, 31, 32, 33, 34, 35], 'text': ['Sample 24', 'Sample 25', '

## Custom modules

### Define module directory

Specify the directory containing the module files with `LAVENDER_DATA_MODULE_DIR` environment variable. Use `.env` file or set the environment variable with `export`.

```sh
export LAVENDER_DATA_MODULE_DIR=/path/to/module/dir
```

We already have a module directory `modules` in the example directory.

```sh
export LAVENDER_DATA_MODULE_DIR=./modules
```

This example module contains a custom filter, collater, and preprocessor. Let's take a look at one by one.


### Online Filters

To online-filter the dataset, define a filter class that inherits from `Filter`. It takes a single sample as an argument and returns a boolean value. If it returns `True`, the sample will be included in the dataset. For example, below is a filter that only includes samples with even `uid`.

In [47]:
from lavender_data.server import FilterRegistry, Filter


@FilterRegistry.register("only_even_uids")
class OnlyEvenUidsFilter(Filter):
    def filter(self, sample: dict) -> bool:
        return sample["uid"] % 2 == 0

On iteration, specify the filter name to use it.

In [48]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
    filter="only_even_uids",
    batch_size=10,
):
    print(sample)

{'text': ['Sample 0', 'Sample 2', 'Sample 4', 'Sample 6', 'Sample 8', 'Sample 10', 'Sample 12', 'Sample 14', 'Sample 16', 'Sample 18'], 'uid': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], '_lavender_data_indices': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]}
{'text': ['Sample 20', 'Sample 22', 'Sample 24', 'Sample 26', 'Sample 28', 'Sample 30', 'Sample 32', 'Sample 34', 'Sample 36', 'Sample 38'], 'uid': [20, 22, 24, 26, 28, 30, 32, 34, 36, 38], '_lavender_data_indices': [20, 22, 24, 26, 28, 30, 32, 34, 36, 38]}
{'text': ['Sample 40', 'Sample 42', 'Sample 44', 'Sample 46', 'Sample 48', 'Sample 50', 'Sample 52', 'Sample 54', 'Sample 56', 'Sample 58'], 'uid': [40, 42, 44, 46, 48, 50, 52, 54, 56, 58], '_lavender_data_indices': [40, 42, 44, 46, 48, 50, 52, 54, 56, 58]}
{'text': ['Sample 60', 'Sample 62', 'Sample 64', 'Sample 66', 'Sample 68', 'Sample 70', 'Sample 72', 'Sample 74', 'Sample 76', 'Sample 78'], 'uid': [60, 62, 64, 66, 68, 70, 72, 74, 76, 78], '_lavender_data_indices': [60, 62, 64, 66, 68, 70, 

### Collater

To collate the samples, define a collater class that inherits from `Collater`. It takes a list of samples as an argument and returns a dictionary of batched samples.

If `torch` is installed, default collater will be `torch.utils.data.default_collate`. If not, it will be a simple function that concatenates the samples to a list, like below.

In [49]:
from lavender_data.server import CollaterRegistry, Collater

@CollaterRegistry.register("pylist")
class PyListCollater(Collater):
    def collate(self, samples: list[dict]) -> dict:
        return {
            "uid": [sample["uid"] for sample in samples],
            "text": [sample["text"] for sample in samples],
        }

In [50]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
    collater="pylist",
    batch_size=10,
):
    print(sample)

{'uid': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'text': ['Sample 0', 'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 8', 'Sample 9'], '_lavender_data_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'uid': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'text': ['Sample 10', 'Sample 11', 'Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16', 'Sample 17', 'Sample 18', 'Sample 19'], '_lavender_data_indices': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
{'uid': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'text': ['Sample 20', 'Sample 21', 'Sample 22', 'Sample 23', 'Sample 24', 'Sample 25', 'Sample 26', 'Sample 27', 'Sample 28', 'Sample 29'], '_lavender_data_indices': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]}
{'uid': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'text': ['Sample 30', 'Sample 31', 'Sample 32', 'Sample 33', 'Sample 34', 'Sample 35', 'Sample 36', 'Sample 37', 'Sample 38', 'Sample 39'], '_lavender_data_indices': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

### Remote Preprocessor

To preprocess the samples remotely, define a preprocessor class that inherits from `Preprocessor`. It takes a collated batch as an argument and returns a preprocessed batch.

In [51]:
from lavender_data.server import PreprocessorRegistry, Preprocessor


@PreprocessorRegistry.register("append_new_column")
class AppendNewColumn(Preprocessor):
    def process(self, batch: dict) -> dict:
        batch["new_column"] = []
        for uid in batch["uid"]:
            batch["new_column"].append(f"{uid}_processed")
        return batch

In [52]:
for sample in Iteration.from_dataset(
    dataset_id=dataset.id,
    shardsets=[shardset.id],
    preprocessor="append_new_column",
    batch_size=10,
):
    print(sample)

{'text': ['Sample 0', 'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 8', 'Sample 9'], 'uid': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'new_column': ['0_processed', '1_processed', '2_processed', '3_processed', '4_processed', '5_processed', '6_processed', '7_processed', '8_processed', '9_processed'], '_lavender_data_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'text': ['Sample 10', 'Sample 11', 'Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16', 'Sample 17', 'Sample 18', 'Sample 19'], 'uid': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'new_column': ['10_processed', '11_processed', '12_processed', '13_processed', '14_processed', '15_processed', '16_processed', '17_processed', '18_processed', '19_processed'], '_lavender_data_indices': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
{'text': ['Sample 20', 'Sample 21', 'Sample 22', 'Sample 23', 'Sample 24', 'Sample 25', 'Sample 26', 'Sample 27', 'Sample 28', 'Sample 29'], 'uid': [20, 21, 22, 23, 24, 25,

## Clean up

Clean up the server process.

In [28]:
server_process.terminate()
server_process.wait()

-15